# Movie Chatbot with vCore Azure Cosmos DB for MongoDB

In this sample, we'll demonstrate how to build a RAG Pattern application using a subset of the Movie Lens dataset. This sample will leverage the SDK's for MongoDB to perform vector search and cache the results. And Azure OpenAI to generate embeddings and LLM completions.

There are two implementations in this project. One using LangChain and this simple implementation. The simple implementation connects directly to Azure Cosmos DB for MongoDB to perform vector search, and cache responses. It also connects directly to Azure OpenAI to generate embeddings and completions. This version requires a user to define and build the LLM payloads for LLM generation and also define the RAG Pattern request pipeline. Cache must be manually consulted in the pipelin and responses must also be manually cached.

The vector search will be done using Azure Cosmos DB for MongoDB's [vector similarity search](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search) functionality to do vector search over the vectorized movie data as well as the conversation history which is also used as a cache.

At the end we will create a simple UX using Gradio to allow users to type in questions and display responses generated by a GPT model or served from the cache. The resopnses will also display an elapsed time so you can see the impact caching has on performance versus generating a response.

**Important Note**: This sample requires you to have a v-Core Azure Cosmos DB for MongoDB account setup. To get started, visit:
- [vCore Azure Cosmos DB for MongoDB Quickstart](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/quickstart-portal)
- [vCore Azure Cosmos DB for MongoDB Vector Search](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search)

# Preliminaries <a class="anchor" id="preliminaries"></a>
First, let's start by installing the packages that we'll need later. 

In [ ]:
! pip install json
! pip install python-dotenv
! pip install pymongo
! pip install openai
! pip install gradio

In [ ]:
import time
import json
import uuid
from dotenv import dotenv_values
import pymongo
from openai import AzureOpenAI
import gradio as gr

Please use the example.env as a template to provide the necessary keys and endpoints in your own .env file.
Make sure to modify the env_name accordingly.

In [60]:
# specify the name of the .env file name 
env_name = "../fabconf.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)
cosmos_conn = config['cosmos_for_mongodb_connection_string']
cosmos_database = config['cosmos_database_name']
cosmos_collection = config['cosmos_collection_name']
cosmos_vector_property = config['cosmos_vector_property_name']
cosmos_cache = config['cosmos_cache_collection_name']

openai_endpoint = config['openai_endpoint']
openai_key = config['openai_key']
openai_api_version = config['openai_api_version']
openai_embeddings_deployment = config['openai_embeddings_deployment']
openai_embeddings_model = config['openai_embeddings_model']
openai_embeddings_dimensions = int(config['openai_embeddings_dimensions'])
openai_completions_deployment = config['openai_completions_deployment']
openai_completions_model = config['openai_completions_model']

In [ ]:
# Create the Azure Cosmos DB for MongoDB client
cosmos_client = pymongo.MongoClient(cosmos_conn)
# Create the OpenAI client
openai_client = AzureOpenAI(azure_endpoint=openai_endpoint, api_key=openai_key, api_version=openai_api_version)

# Database and collections

In [ ]:
# Get the database
database = cosmos_client[cosmos_database]

# Get the movie collection
movies_container = database[cosmos_collection]

# Get the cache collection
cache_container = database[cosmos_cache]

# Generate embeddings from Azure OpenAI

This is used to vectorize the user input for the vector search

In [ ]:
def generate_embeddings(text):
    response = openai_client.embeddings.create(
        input = text, 
        model = openai_embeddings_deployment, 
        dimensions = openai_embeddings_dimensions)

    embeddings = response.model_dump()
    return embeddings['data'][0]['embedding']

# Vector Search in Azure Cosmos DB for MongoDB

This defines a function for performing a vector search over any collection in Azure Cosmos DB for MongoDB. Function takes a collection reference, array of vectors, and optional similarity score to filter for top matches and number of results to return to filter further.

In [ ]:
def vector_search(collection, vectors, similarity_score=0.02, num_results=3):
    
    pipeline = [
        {
        '$search': {
            "cosmosSearch": {
                "vector": vectors,
                "path": cosmos_vector_property,
                "k": num_results,
                "efsearch": 40 # optional for HNSW only 
            },
            "returnStoredSource": True }},
            { '$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } },
            { '$match': { "similarityScore": { '$gt': similarity_score } } 
        }   
    ]

    results = list(collection.aggregate(pipeline))

    # Exclude the 'vector' to reduce payload size to LLM and _id properties to avoid serialization issues 
    for result in results:
        del result['document']['vector']
        del result['_id']
        del result['document']['_id']
    
    return results

# Get recent chat history

This function provides conversational context to the LLM, allowing it to better have a conversation with the user.

In [ ]:
# Grab chat history to as part of the payload to GPT model for completion.
def get_chat_history(container,completions=3):

    # Sort by _id in descending order and limit the results to the completions value passed in
    results = container.find({}, {"prompt": 1, "completion": 1}).sort([("_id", -1)]).limit(completions)
    return results

# Chat Completion Function

This function assembles all of the required data as a payload to send to a GPT model to generate a completion

In [ ]:
def generate_completion(user_prompt, vector_search_results, chat_history):
    
    system_prompt = '''
    You are an intelligent assistant for movies. You are designed to provide helpful answers to user questions about movies in your database.
    You are friendly, helpful, and informative and can be lighthearted. Be concise in your responses, but still friendly.
        - Only answer questions related to the information provided below. Provide at least 3 candidate movie answers in a list.
        - Write two lines of whitespace between each answer in the list.
    '''
    
    # Create a list of messages as a payload to send to the OpenAI Completions API

    # system prompt
    messages = [{'role': 'system', 'content': system_prompt}]

    #chat history
    for chat in chat_history:
        messages.append({'role': 'user', 'content': chat['prompt'] + " " + chat['completion']})
    
    #user prompt
    messages.append({'role': 'user', 'content': user_prompt})

    #vector search results
    for result in vector_search_results:
        messages.append({'role': 'system', 'content': json.dumps(result['document'])})

    # Create the completion
        response = openai_client.chat.completions.create(
            model = openai_completions_deployment,
            messages = messages 
        )
    
    return response.model_dump()

# Cache Generated Responses

Save the user prompts and generated completions in a conversation. Used to answer the same questions from other users. This is cheaper and faster than generating results each time.

In [ ]:
def cache_response(container,user_prompt, prompt_vectors, response):
    # Create a dictionary representing the chat document
    chat_document = {
        'id':  str(uuid.uuid4()),  
        'prompt': user_prompt,
        'completion': response['choices'][0]['message']['content'],
        'completionTokens': str(response['usage']['completion_tokens']),
        'promptTokens': str(response['usage']['prompt_tokens']),
        'totalTokens': str(response['usage']['total_tokens']),
        'model': response['model'],
        'vector': prompt_vectors
    }
    # Insert the chat document into the Cosmos DB container
    container.insert_one(chat_document)
    print("item inserted into cache.", chat_document)

# LLM Pipeline function

This function defines the pipeline for our RAG Pattern application. When user submits a question, the cache is consulted first for an exact match. If no match then a vector search is made, chat history gathered, the LLM generates a response, which is then cached before returning to the user.

In [ ]:
def chat_completion(cache_container, movies_container, user_input):
    print("starting completion")
    # Generate embeddings from the user input
    user_embeddings = generate_embeddings(user_input)
    # Query the chat history cache first to see if this question has been asked before
    cache_results = vector_search(cache_container, vectors = user_embeddings, similarity_score=0.99, num_results=1)
    if len(cache_results) > 0:
        print(cache_results)
        print("Cached Result\n")
        return cache_results[0]['document']['completion'], True
        
    else:
        #perform vector search on the movie collection
        print("New result\n")
        search_results = vector_search(movies_container, user_embeddings)

        print("Getting Chat History\n")
        #chat history
        chat_history = get_chat_history(cache_container, 3)
        #generate the completion
        print("Generating completions \n")
        completions_results = generate_completion(user_input, search_results, chat_history)

        print("Caching response \n")
        #cache the response
        cache_response(cache_container, user_input, user_embeddings, completions_results)

        print("\n")
        # Return the generated LLM completion
        return completions_results['choices'][0]['message']['content'], False


# Create a simple UX in Gradio


In [ ]:
chat_history = []
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Cosmic Movie Assistant")
    
    msg = gr.Textbox(label="Ask me about movies in the Cosmic Movie Database!")
    clear = gr.Button("Clear")

    def user(user_message, chat_history, pk):
        # Create a timer to measure the time it takes to complete the request
        start_time = time.time()
        # Get LLM completion
        response_payload, cached = chat_completion(cache_container, movies_container, user_message)
        # Stop the timer
        end_time = time.time()
        elapsed_time = round((end_time - start_time) * 1000, 2)
        response = response_payload
        print(response_payload)
        # Append user message and response to chat history
        details = f"\n (Time: {elapsed_time}ms)"
        if cached:
            details += " (Cached)"
        chat_history.append([user_message, response_payload + details])
        
        return gr.update(value=""), chat_history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)

    clear.click(lambda: None, None, chatbot, queue=False)

In [ ]:
# launch the gradio interface
demo.launch(debug=True)

In [ ]:
# be sure to run this cell to close or restart the gradio demo
demo.close()